In [3]:
# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
import skimage.transform

# import custom modules
code_dir = '/user_data/mmhender/imStat/code/'
sys.path.append(code_dir)
from feature_extraction import texture_statistics_gabor, texture_statistics_pyramid, bdcn_features, sketch_token_features
from utils import nsd_utils, roi_utils

bdcn_path = '/user_data/mmhender/toolboxes/BDCN/'

from model_fitting import initialize_fitting, merge_features, fwrf_fit, fwrf_predict

fpX = np.float32
device = 'cpu:0'
# device = initialize_fitting.init_cuda()

In [4]:
fitting_type='gabor_texture'
do_avg_pool=True
subject=1
volume_space = True
up_to_sess = 1
n_ori = 4
n_sf = 4
nonlin_fn = False
padding_mode = 'circular';
group_all_hl_feats = True; \
sample_batch_size = 50; voxel_batch_size = 100; \
zscore_features = True; ridge = True; \
shuffle_images = False; random_images = False; random_voxel_data = False; \
do_fitting = True; do_val = True; do_varpart = True; date_str = None;
shuff_rnd_seed = 0; debug = True; \
do_pca = False; min_pct_var = 99; max_pc_to_retain = 400; map_ind = -1; \
n_prf_sd_out = 2; mult_patch_by_prf = True; \
downsample_factor = 1.0; do_nms = False

In [5]:
if 'pyramid' in fitting_type:
    model_name = initialize_fitting.get_pyramid_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = []
        
    if 'plus_bdcn' in fitting_type:
        model_name2 = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)
        model_name = model_name + '_plus_' + model_name2

elif 'bdcn' in fitting_type:
    model_name = initialize_fitting.get_bdcn_model_name(do_pca, map_ind)

elif 'sketch_tokens' in fitting_type:
    model_name = initialize_fitting.get_sketch_tokens_model_name(do_pca)
     
elif 'gabor_texture' in fitting_type:

    model_name = initialize_fitting.get_gabor_texture_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = []

elif 'gabor_solo' in fitting_type:

    model_name = initialize_fitting.get_gabor_solo_model_name(ridge, n_ori, n_sf)
    feature_types_exclude = ['pixel', 'simple_feature_means', 'autocorrs', 'crosscorrs']
else:     
    raise ValueError('your string for fitting_type was not recognized')

output_dir, fn2save = initialize_fitting.get_save_path(subject, volume_space, model_name, shuffle_images, random_images, random_voxel_data, debug, date_str)

# decide what voxels to use  
voxel_mask, voxel_index, voxel_roi, voxel_ncsnr, brain_nii_shape = roi_utils.get_voxel_roi_info(subject, volume_space)

# sessions = np.arange(0,up_to_sess)
sessions = [5,8]
zscore_betas_within_sess = True
# get all data and corresponding images, in two splits. always fixed set that gets left out
trn_stim_data, trn_voxel_data, val_stim_data, val_voxel_data, \
            image_order, image_order_trn, image_order_val = nsd_utils.get_data_splits(subject, sessions=sessions, \
                                                                         voxel_mask=voxel_mask, volume_space=volume_space, \
                                                                          zscore_betas_within_sess=zscore_betas_within_sess, \
                                                                          shuffle_images=shuffle_images, random_images=random_images, \
                                                                                             random_voxel_data=random_voxel_data)
    

Time Stamp: Sep-19-2021_1203_15

Will save final output file to /user_data/mmhender/imStat/model_fits/S01/texture_gabor_ridge_4ori_4sf/Sep-19-2021_1203_15_DEBUG/


Volume space: ROI defs are located at: /lab_data/tarrlab/common/datasets/NSD/nsddata/ppdata/subj01/func1pt8mm/roi

3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[-1.  0. 26. 27. 28. 30. 31. 32. 33.]
1535 voxels are defined (differently) in both retinotopic areas and category areas

14913 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,

In [18]:
len(image_order_trn.shape)

1

In [4]:
if 'sketch_tokens' in fitting_type:
        # For this model, the features are pre-computed, so we will just load them rather than passing in images.
        # Going to pass the image indices (into 10,000 dim array) instead of images to fitting and val functions, 
        # which will tell which features to load.
        trn_stim_data = image_order_trn
        val_stim_data = image_order_val
        
# More params for fitting
holdout_size, lambdas = initialize_fitting.get_fitting_pars(trn_voxel_data, zscore_features, ridge=ridge)
# Params for the spatial aspect of the model (possible pRFs)
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    
    


Possible lambda values are:
[1.0000000e+00 4.2169652e+00 1.7782795e+01 7.4989418e+01 3.1622775e+02
 1.3335215e+03 5.6234131e+03 2.3713736e+04 1.0000000e+05]
most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]


In [5]:

map_resolution = 227
_feature_extractor = sketch_token_features.sketch_token_feature_extractor(subject, device, map_resolution=map_resolution, aperture = aperture, \
                                                     n_prf_sd_out = n_prf_sd_out, \
                                     batch_size=sample_batch_size, mult_patch_by_prf=mult_patch_by_prf, do_avg_pool = do_avg_pool, \
                                     do_pca = do_pca, min_pct_var = min_pct_var, max_pc_to_retain = max_pc_to_retain)


In [7]:
_feature_extractor.forward

<bound method sketch_token_feature_extractor.forward of sketch_token_feature_extractor()>

In [6]:
# add an intercept
add_bias=True
# determines whether to shuffle before separating the nested heldout data for lambda and param selection. 
# always using true.
shuffle=True 
sample_batch_size=100
best_losses, best_lambdas, best_params = fwrf_fit.fit_fwrf_model(trn_stim_data, trn_voxel_data, _feature_extractor, models, \
                                               lambdas, zscore=zscore_features, add_bias=add_bias, \
                                               voxel_batch_size=voxel_batch_size, holdout_size=holdout_size, \
                                               shuffle=shuffle, shuff_rnd_seed=shuff_rnd_seed, device=device, \
                                               dtype=fpX, debug=debug)


dtype = <class 'numpy.float32'>
device = cpu:0
trn_size = 1210 (90.0%)
Computing a new random seed
Seeding random number generator: seed is 421216
Initializing for fitting
Clearing features from memory
---------------------------------------


Getting features for prf 0: [x,y,sigma] is [-0.55 -0.55 0.0400]
Loading pre-computed features from /user_data/mmhender/features/sketch_tokens/S1_features_each_prf.h5py
Took 15.11594 seconds to load file
Size of features array for this image set is:
(1345, 151, 875)
Final size of feature matrix is:
(1345, 151)

Fitting version 0 of 1: full_model, 
fitting model    0 of 875 , voxels [ 14900:14912 ] of 14913
Getting features for prf 1: [x,y,sigma] is [-0.49 -0.55 0.0400]
Final size of feature matrix is:
(1345, 151)

Fitting version 0 of 1: full_model, 
fitting model    1 of 875 , voxels [ 14900:14912 ] of 14913
---------------------------------------
total time = 20.116570s
total throughput = 0.001349s/voxel
voxel throughput = 0.000271s/voxel
setup 

In [13]:
zscore_features

True

In [7]:
best_losses

array([[139.42769 ],
       [153.0108  ],
       [113.51092 ],
       ...,
       [128.69508 ],
       [114.174614],
       [103.69032 ]], dtype=float32)

In [11]:
best_params[1]

array([[[-5.9103139e-11],
        [ 9.1786148e-11],
        [-4.4165206e-11],
        ...,
        [-7.2541209e-11],
        [-8.0176137e-11],
        [ 1.3179347e-09]],

       [[-1.0445406e-07],
        [ 2.1558332e-08],
        [-8.1119353e-08],
        ...,
        [-1.1369220e-07],
        [-1.0098479e-07],
        [ 9.6808626e-06]],

       [[ 2.6421698e-11],
        [ 9.7884999e-12],
        [ 4.3789229e-11],
        ...,
        [ 4.3280674e-12],
        [ 4.8803218e-11],
        [-7.7500140e-09]],

       ...,

       [[-4.3056489e-11],
        [-9.8471530e-11],
        [-2.9640321e-11],
        ...,
        [-6.0766954e-11],
        [-4.7329831e-11],
        [ 1.1012556e-08]],

       [[ 7.0764475e-12],
        [ 7.0934821e-11],
        [ 7.0353185e-12],
        ...,
        [-3.5245234e-11],
        [ 1.3029000e-11],
        [-7.3201183e-09]],

       [[ 1.6755065e-06],
        [-8.1694616e-06],
        [-1.4743752e-06],
        ...,
        [ 4.5739307e-06],
        [-2.787